# Pipeline for locally accessing Inpainting API by stability AI.

## 0. Install Required Libraries

pip install pillow requests ipython

## 1. Import Required Libraries

In [1]:
from io import BytesIO
import json
import os
from PIL import Image
import requests
import time
from IPython.display import display

## 2. Connect to stability API

In [2]:
import getpass
# To get your API key, message me on slack to receive it...
STABILITY_KEY = getpass.getpass('Enter your API Key: ')


## 3. Define Helper Functions

In [4]:
def send_generation_request(host, params):
    headers = {
        "Accept": "image/*",
        "Authorization": f"Bearer {STABILITY_KEY}"
    }

    # Encode parameters
    files = {}
    image = params.pop("image", None)
    mask = params.pop("mask", None)
    if image:
        files["image"] = open(image, 'rb')
    if mask:
        files["mask"] = open(mask, 'rb')
    if not files:
        files["none"] = ''

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    return response

def send_async_generation_request(host, params):
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {STABILITY_KEY}"
    }

    # Encode parameters
    files = {}
    if "image" in params:
        image = params.pop("image")
        files = {"image": open(image, 'rb')}

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    # Process async response
    response_dict = response.json()
    generation_id = response_dict.get("id")
    if not generation_id:
        raise Exception("Expected 'id' in response")

    # Loop until result or timeout
    timeout = int(os.getenv("WORKER_TIMEOUT", 500))
    start = time.time()
    status_code = 202
    while status_code == 202:
        response = requests.get(
            f"{host}/result/{generation_id}",
            headers={
                **headers,
                "Accept": "image/*"
            },
        )

        if not response.ok:
            raise Exception(f"HTTP {response.status_code}: {response.text}")
        status_code = response.status_code
        time.sleep(10)
        if time.time() - start > timeout:
            raise Exception(f"Timeout after {timeout} seconds")

    return response


## 4. Specify Paths and Prompts

In [5]:
input_folder = "D:\Data Downloads\Bing Image Scraped Results\Korean_Clothes"
mask_folder = "D:\Data Downloads\Bing Image Scraped Results\Korean_Clothes_Mask"
output_folder = "D:\Data Downloads\Bing Image Scraped Results\Korean_Clothes_Synthesized\White"
prompt = "White Person Wearing the Clothes"
negative_prompt = ""
seed = 0
output_format = "png"

## 5. Perform Inpainting

In [ ]:
import os
from PIL import Image

def process_images(input_folder, mask_folder, output_folder, prompt, negative_prompt="", seed=0, output_format="png"):
    # Maximum allowed dimensions
    MAX_PIXELS = 9437184  # 3072 x 3072 pixels
    
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Iterate through all files in the input folder
    for image_file in os.listdir(input_folder):
        # Construct the paths for the input image and the corresponding mask
        image_path = os.path.join(input_folder, image_file)
        mask_path = os.path.join(mask_folder, os.path.splitext(image_file)[0] + "_mask.png")  # Assuming mask has same name with "_mask" suffix
        
        # Skip if mask doesn't exist
        if not os.path.exists(mask_path):
            print(f"Mask not found for {image_file}, skipping...")
            continue
        
        # Open the image to check dimensions
        with Image.open(image_path) as img:
            width, height = img.size
            if width * height > MAX_PIXELS:
                scale_factor = (MAX_PIXELS / (width * height)) ** 0.5
                new_width = int(width * scale_factor)
                new_height = int(height * scale_factor)
                img = img.resize((new_width, new_height), Image.ANTIALIAS)
                
                # Save the resized image temporarily
                resized_image_path = os.path.join(output_folder, f"resized_{image_file}")
                img.save(resized_image_path)
                print(f"Resized {image_file} to {new_width}x{new_height}")
            else:
                resized_image_path = image_path  # No resizing needed

        # Parameters for the API
        params = {
            "image": resized_image_path,
            "mask": mask_path,
            "negative_prompt": negative_prompt,
            "seed": seed,
            "mode": "mask",
            "output_format": output_format,
            "prompt": prompt
        }
        
        # Call the API
        response = send_generation_request("https://api.stability.ai/v2beta/stable-image/edit/inpaint", params)
        
        # Decode response
        output_image = response.content
        finish_reason = response.headers.get("finish-reason")
        seed = response.headers.get("seed")
        
        # Check for NSFW classification
        if finish_reason == 'CONTENT_FILTERED':
            print(f"Generation for {image_file} failed due to NSFW classification.")
            continue
        
        # Save the result image
        filename, _ = os.path.splitext(image_file)
        edited_filename = f"edited_{filename}_{seed}.{output_format}"
        edited_path = os.path.join(output_folder, edited_filename)
        
        with open(edited_path, "wb") as f:
            f.write(output_image)
        
        print(f"Saved image to {edited_path}")

# Process images
process_images(input_folder, mask_folder, output_folder, prompt, negative_prompt, seed, output_format)


Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
Saved image to D:\Data Downloads\Bing Image Scraped Results\Korean_Clothes_Synthesized\White\edited_01008_725168278.png
Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
